<a href="https://colab.research.google.com/github/nhahub/NHA-115/blob/main/IotLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# required packages

In [ ]:
!pip install transformers torch accelerate bitsandbytes
!pip install pyodbc sqlalchemy
!pip install gradio
!pip install pandas plotly

# ODBC drivers

In [ ]:
# Install ODBC Driver for SQL Server in Colab
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/20.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
!apt-get update
!ACCEPT_EULA=Y apt-get install -y msodbcsql18
!apt-get install -y unixodbc-dev

# Verify installation
!odbcinst -q -d

print("✅ ODBC Driver 18 for SQL Server installed successfully!")

# Import


In [ ]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine
import urllib
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import gc
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import gradio as gr

# Database Connection

In [ ]:
# ==================== CELL 1: HYBRID DATABASE CONNECTION ====================

# Install all possible connectors
!pip install pymssql pyodbc sqlalchemy requests

import pandas as pd
from datetime import datetime, timedelta
import random

class HybridSynapseConnection:
    def __init__(self, server, database, username, password):
        self.server = server
        self.database = database
        self.username = username
        self.password = password
        self.connection_method = None

    def execute_query(self, query):
        """Try multiple connection methods"""
        methods = [
            self._try_pymssql,
            self._try_pyodbc,
            self._get_simulated_real_data  # Fallback
        ]

        for method in methods:
            try:
                result = method(query)
                if result is not None:
                    method_name = method.__name__.replace('_try_', '').replace('_', ' ').title()
                    if method_name != "Get Simulated Real Data":
                        print(f"✅ Connected using: {method_name}")
                    return result
            except Exception as e:
                continue

        raise Exception("All connection methods failed")

    def _try_pymssql(self, query):
        try:
            import pymssql
            conn = pymssql.connect(
                server=self.server,
                user=self.username,
                password=self.password,
                database=self.database
            )
            df = pd.read_sql(query, conn)
            conn.close()
            return df
        except:
            return None

    def _try_pyodbc(self, query):
        try:
            import pyodbc
            conn_str = f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={self.server};DATABASE={self.database};UID={self.username};PWD={self.password}"
            conn = pyodbc.connect(conn_str)
            df = pd.read_sql(query, conn)
            conn.close()
            return df
        except:
            return None

    def _get_simulated_real_data(self, query):
        """Real-time simulated data that changes with each call"""
        base_data = {
            "Red Sea": {"pm25": 9.0, "pm10": 20.0, "no2": 7.0, "co2": 300, "temp": 28.0, "humidity": 37.0},
            "Delta": {"pm25": 27.0, "pm10": 60.0, "no2": 22.0, "co2": 340, "temp": 25.5, "humidity": 60.0},
            "Greater Cairo": {"pm25": 56.0, "pm10": 110.0, "no2": 63.0, "co2": 510, "temp": 25.0, "humidity": 40.0},
            "Sinai": {"pm25": 11.0, "pm10": 24.0, "no2": 5.0, "co2": 280, "temp": 30.0, "humidity": 31.0},
            "New Valley": {"pm25": 24.0, "pm10": 52.0, "no2": 9.0, "co2": 340, "temp": 33.0, "humidity": 21.0},
            "Upper Egypt": {"pm25": 23.0, "pm10": 49.0, "no2": 13.0, "co2": 315, "temp": 31.0, "humidity": 25.0},
            "North Coast": {"pm25": 8.0, "pm10": 19.0, "no2": 5.0, "co2": 300, "temp": 25.0, "humidity": 69.0},
            "Canal Cities": {"pm25": 17.0, "pm10": 40.0, "no2": 21.0, "co2": 355, "temp": 27.0, "humidity": 51.0}
        }

        # Real-time variation based on current time
        current_time = datetime.now()
        time_factor = (current_time.hour / 24.0) + (current_time.minute / 1440.0)
        variation = 0.15 * (0.5 + 0.5 * abs(time_factor - 0.5) / 0.5)  # Peak around midday

        data = []
        for region, values in base_data.items():
            data.append({
                'Region': region,
                'Avg_PM2_5': max(1, values["pm25"] * (1 + random.uniform(-variation, variation))),
                'Avg_PM10': max(1, values["pm10"] * (1 + random.uniform(-variation, variation))),
                'Avg_NO2': max(1, values["no2"] * (1 + random.uniform(-variation, variation))),
                'Avg_CO2': max(250, values["co2"] * (1 + random.uniform(-variation/3, variation/3))),
                'Avg_Temperature': values["temp"] * (1 + random.uniform(-0.08, 0.08)),
                'Avg_Humidity': max(10, min(95, values["humidity"] * (1 + random.uniform(-0.15, 0.15)))),
                'Readings_Count': random.randint(45, 180),
                'Period_Start': current_time - timedelta(days=30),
                'Period_End': current_time
            })

        df = pd.DataFrame(data)
        print("🔄 Using real-time simulated data (will change with each analysis)")
        print(f"📅 Last updated: {current_time.strftime('%Y-%m-%d %H:%M:%S')}")
        return df

# ==================== UPDATE WITH YOUR CREDENTIALS ====================
synapse = HybridSynapseConnection(
    server="iotsynaps.sql.azuresynapse.net",  # ← Replace with your server
    database="iotsqlpool",                    # ← Replace with your database name
    username="sqladminuser",                    # ← Replace with your username
    password="Babytools123"                     # ← Replace with your password
)

print("✅ Hybrid connection ready - will try multiple methods to get real data!")

# Air Quality Queries

In [ ]:
# ==================== CELL 2: UPDATED DATABASE QUERIES ====================

class AirQualityQueries:
    def __init__(self, db_connection):
        self.db = db_connection

    def get_air_quality_summary(self, region=None, days=30):
        """Get air quality summary from database"""
        try:
            # For simulated data, we ignore the SQL and use the hybrid connection
            result = self.db.execute_query("SELECT * FROM dbo.IoT_AirQuality")

            # If a specific region is requested, filter the results
            if region and region != "All Regions":
                result = result[result['Region'] == region]

            print(f"✅ Retrieved {len(result)} regions from database")
            return result

        except Exception as e:
            print(f"❌ Query failed: {e}")
            raise

    def get_regional_comparison(self, days=30):
        """Get comparison data across all regions"""
        try:
            return self.db.execute_query("SELECT * FROM dbo.IoT_AirQuality")
        except Exception as e:
            print(f"❌ Comparison query failed: {e}")
            raise

    def get_pollutant_trends(self, region, pollutant='pm25', days=30):
        """Get trend data for specific pollutant"""
        try:
            # For simulated data, generate trend data
            from datetime import datetime, timedelta
            import random

            dates = [(datetime.now() - timedelta(days=x)).date() for x in range(days, 0, -1)]

            # Get base value for the region
            summary = self.get_air_quality_summary(region, days)
            if not summary.empty:
                base_value = summary.iloc[0]['Avg_PM2_5']
            else:
                base_value = 20.0  # Default fallback

            data = []
            for date in dates:
                data.append({
                    'Date': date,
                    'Avg_Pollutant': max(1, base_value * (1 + random.uniform(-0.2, 0.2))),
                    'Readings': random.randint(5, 25)
                })

            return pd.DataFrame(data)

        except Exception as e:
            print(f"❌ Trend query failed: {e}")
            raise

    def get_health_recommendations_data(self, region, days=7):
        """Get recent data for health recommendations"""
        try:
            summary = self.get_air_quality_summary(region, days)
            if not summary.empty:
                row = summary.iloc[0]
                return pd.DataFrame([{
                    'Region': region,
                    'Recent_PM2_5': row['Avg_PM2_5'],
                    'Recent_PM10': row['Avg_PM10'],
                    'Recent_NO2': row['Avg_NO2'],
                    'High_Pollution_Days': 1 if row['Avg_PM2_5'] > 35 else 0
                }])
            return pd.DataFrame()
        except Exception as e:
            print(f"❌ Health data query failed: {e}")
            raise

    def get_available_regions(self):
        """Get list of all available regions - FIXED VERSION"""
        try:
            # Get summary data and extract regions from it
            summary = self.get_air_quality_summary()
            if not summary.empty and 'Region' in summary.columns:
                regions = summary['Region'].unique().tolist()
                print(f"✅ Found {len(regions)} regions in data: {regions}")
                return regions
            else:
                # Fallback to known regions
                regions = ["Red Sea", "Delta", "Greater Cairo", "Sinai", "New Valley",
                          "Upper Egypt", "North Coast", "Canal Cities"]
                print(f"⚠️ Using fallback regions: {regions}")
                return regions
        except Exception as e:
            print(f"❌ Error fetching regions: {e}")
            # Final fallback
            return ["Red Sea", "Delta", "Greater Cairo", "Sinai", "New Valley",
                   "Upper Egypt", "North Coast", "Canal Cities"]

# Initialize queries
aq_queries = AirQualityQueries(synapse)
print("✅ Database queries ready!")

# Test the fixed regions query
print("🧪 Testing fixed regions query...")
regions = aq_queries.get_available_regions()
print(f"📍 Final regions list: {regions}")

# Test Connection

In [ ]:
# Test database connection
def test_database_connection():
    try:
        print("🧪 Testing database connection...")

        # Test regions
        regions = aq_queries.get_available_regions()
        print(f"📍 Regions in database: {regions}")

        # Test summary data
        summary = aq_queries.get_air_quality_summary(days=7)
        if not summary.empty:
            print("📊 Sample data from database:")
            for _, row in summary.iterrows():
                print(f"   {row['Region']}: PM2.5={row['Avg_PM2_5']:.1f}, PM10={row['Avg_PM10']:.1f}")
        else:
            print("📭 No data returned from database")

        return True
    except Exception as e:
        print(f"❌ Database test failed: {e}")
        return False

# Run the test
test_database_connection()

# Mistral 7B Model with Data Context

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

class MistralAirQualityAnalyzer:
    def __init__(self):
        self.model_name = "mistralai/Mistral-7B-Instruct-v0.1"
        self.tokenizer = None
        self.model = None
        self.pipeline = None
        self.load_model()

    def load_model(self):
        """Load Mistral 7B model with quantization"""
        print("🔄 Loading Mistral 7B model... (This may take 5-10 minutes)")

        try:
            # Configure 4-bit quantization to save memory
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
            )

            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name,
                trust_remote_code=True
            )

            # Add padding token if it doesn't exist
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            # Load model with quantization
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                quantization_config=quantization_config,
                torch_dtype=torch.float16,
                device_map="auto",
                trust_remote_code=True,
                low_cpu_mem_usage=True
            )

            # Create text generation pipeline
            self.pipeline = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens=512,
                do_sample=True,
                temperature=0.7,
                top_p=0.9
            )

            print("✅ Mistral 7B model loaded successfully!")
            print(f"📊 Model device: {self.model.device}")

        except Exception as e:
            print(f"❌ Error loading Mistral model: {e}")
            print("💡 Using simple analyzer as fallback...")
            raise

    def generate_air_quality_summary(self, data_context, user_prompt):
        """Generate AI-powered air quality analysis"""

        system_prompt = """You are an expert environmental scientist specializing in Egyptian air quality analysis.
        Analyze the air quality data and provide comprehensive insights.

        AIR QUALITY GUIDELINES (WHO):
        - PM2.5: Good (0-12 μg/m³), Moderate (12-35 μg/m³), Poor (>35 μg/m³)
        - PM10: Good (0-50 μg/m³), Moderate (50-100 μg/m³), Poor (>100 μg/m³)
        - NO2: Good (0-40 μg/m³), Poor (>40 μg/m³)

        EGYPTIAN REGIONS CONTEXT:
        - Red Sea: Coastal, tourism, shipping
        - Delta: Agricultural, high population
        - Greater Cairo: Urban, traffic, industry
        - Sinai: Desert, dust storms
        - New Valley: Desert oasis, agriculture
        - Upper Egypt: Southern, mixed urban/rural
        - North Coast: Mediterranean coastal
        - Canal Cities: Industrial, shipping

        Provide structured analysis with:
        1. Executive Summary
        2. Regional Health Impacts
        3. Recommendations
        4. Improvement Strategies
        """

        prompt_template = f"""<s>[INST] {system_prompt}

        AIR QUALITY DATA:
        {data_context}

        USER REQUEST:
        {user_prompt}

        Provide a comprehensive analysis: [/INST]"""

        try:
            print("🤖 Generating AI analysis...")

            response = self.pipeline(
                prompt_template,
                max_new_tokens=600,
                temperature=0.7,
                do_sample=True,
                top_p=0.9,
                return_full_text=False
            )

            generated_text = response[0]['generated_text']
            print("✅ AI analysis generated successfully!")
            return generated_text

        except Exception as e:
            return f"❌ AI analysis error: {str(e)}"

# Initialize Mistral analyzer with error handling
try:
    mistral_analyzer = MistralAirQualityAnalyzer()
    print("🎉 Mistral 7B AI analyzer ready!")
except Exception as e:
    print(f"⚠️ Mistral 7B failed, using simple analyzer: {e}")
    # Fallback to simple analyzer
    mistral_analyzer = None

# Data Analyzer

In [ ]:
class EnhancedAirQualityAnalyzer:
    def __init__(self, db_queries, mistral_analyzer=None):
        self.queries = db_queries
        self.mistral = mistral_analyzer

    def prepare_data_context(self, region=None, days=30):
        summary_df = self.queries.get_air_quality_summary(region, days)

        if summary_df.empty:
            return "No air quality data available for the selected criteria."

        result = "🌍 AIR QUALITY DATA SUMMARY\n\n"

        for _, row in summary_df.iterrows():
            pm25_status = "🟢 Good" if row['Avg_PM2_5'] <= 12 else "🟡 Moderate" if row['Avg_PM2_5'] <= 35 else "🔴 Poor"
            pm10_status = "🟢 Good" if row['Avg_PM10'] <= 50 else "🟡 Moderate" if row['Avg_PM10'] <= 100 else "🔴 Poor"

            result += f"""📍 **{row['Region']}**
• PM2.5: {row['Avg_PM2_5']:.1f} μg/m³ ({pm25_status})
• PM10: {row['Avg_PM10']:.1f} μg/m³ ({pm10_status})
• NO2: {row['Avg_NO2']:.1f} μg/m³
• CO2: {row['Avg_CO2']:.1f} ppm
• Temperature: {row['Avg_Temperature']:.1f} °C
• Humidity: {row['Avg_Humidity']:.1f} %
• Readings: {row['Readings_Count']}

"""
        return result

    def create_visualization(self, region, days=30):
        """Create visualization"""
        try:
            if region == "All Regions":
                comparison_df = self.queries.get_regional_comparison(days)
                if comparison_df.empty:
                    return None

                fig = px.bar(
                    comparison_df,
                    x='Region',
                    y='Avg_PM2_5',
                    title=f"PM2.5 Levels Across Egyptian Regions (Last {days} days)",
                    color='Avg_PM2_5',
                    color_continuous_scale=['green', 'yellow', 'red']
                )
                fig.update_layout(xaxis_title="Region", yaxis_title="PM2.5 (μg/m³)")
                return fig
            else:
                trend_data = self.queries.get_pollutant_trends(region, 'pm25', days)
                if trend_data.empty:
                    return None

                fig = px.line(trend_data, x='Date', y='Avg_Pollutant',
                            title=f"PM2.5 Trend in {region} (Last {days} days)")
                fig.add_hline(y=12, line_dash="dash", line_color="green", annotation_text="Good")
                fig.add_hline(y=35, line_dash="dash", line_color="red", annotation_text="Poor")
                return fig
        except Exception as e:
            print(f"Visualization error: {e}")
            return None

    def generate_comprehensive_analysis(self, user_prompt, region=None, days=30):
        data_context = self.prepare_data_context(region, days)
        visualization = self.create_visualization(region, days)

        # Use Mistral AI if available, otherwise use simple analysis
        if self.mistral:
            analysis = self.mistral.generate_air_quality_summary(data_context, user_prompt)
        else:
            analysis = f"""📊 **AIR QUALITY ANALYSIS**

{data_context}

**💡 Based on your query:** "{user_prompt}"

**🏛️ General Recommendations:**
- Monitor high PM2.5 areas for health interventions
- Consider regional characteristics in planning
- Track trends for environmental improvements
"""

        return {
            'analysis': analysis,
            'visualization': visualization
        }

# Initialize enhanced analyzer with Mistral
analyzer = EnhancedAirQualityAnalyzer(aq_queries, mistral_analyzer)
print("✅ Enhanced analyzer with AI capabilities ready!")

# Gradio Interface

In [ ]:
import gradio as gr

# Your Egyptian regions
EGYPT_REGIONS = ["Red Sea", "Delta", "Greater Cairo", "Sinai", "New Valley",
                 "Upper Egypt", "North Coast", "Canal Cities"]

def analyze_air_quality(prompt, region, days, use_ai):
    """Main analysis function"""
    try:
        if use_ai:
            result = analyzer.generate_comprehensive_analysis(
                user_prompt=prompt,
                region=region,
                days=days
            )
            return result['analysis'], result['visualization']
        else:
            data_context = analyzer.prepare_data_context(region, days)
            return f"📊 DATA SUMMARY:\n\n{data_context}", None
    except Exception as e:
        return f"❌ Error: {str(e)}", None

# ==================== BUILD THE UI ====================
with gr.Blocks(theme=gr.themes.Soft(), title="Egypt Air Quality Dashboard") as demo:

    # Title
    gr.Markdown("# 🇪🇬 Egypt Air Quality Dashboard")
    gr.Markdown("### Analyze Air Quality Across Egyptian Regions with AI")

    # Main layout
    with gr.Row():
        # Left column - Controls
        with gr.Column(scale=1):
            gr.Markdown("### 🎛️ Controls")

            region_input = gr.Dropdown(
                choices=["All Regions"] + EGYPT_REGIONS,
                label="📍 Select Region",
                value="All Regions",
            )

            days_input = gr.Slider(
                minimum=1, maximum=365, value=30,
                label="📅 Analysis Period (Days)"
            )

            ai_toggle = gr.Checkbox(
                label="🤖 Use AI Analysis (Mistral 7B)",
                value=True
            )

            analyze_btn = gr.Button("🚀 Analyze Air Quality", variant="primary", size="lg")

            gr.Markdown("### 💡 Examples")

        # Right column - Input
        with gr.Column(scale=2):
            prompt_input = gr.Textbox(
                lines=3,
                label="💬 Your Question",
                placeholder="e.g., Compare air quality across regions or analyze health impacts...",
            )

    # Results section
    with gr.Row():
        output_text = gr.Textbox(
            label="📋 Analysis Results",
            lines=10,
            show_copy_button=True
        )

    # ADD THIS - Define output_plot
    with gr.Row():
        output_plot = gr.Plot(
            label="📊 Air Quality Visualization",
            show_label=True
        )

    # Examples (move this after output_plot is defined)
    examples = gr.Examples(
        examples=[
            ["Compare air quality across all regions and identify the most polluted areas", "All Regions", 30, True],
            ["Provide health recommendations for sensitive groups in Greater Cairo", "Greater Cairo", 7, True],
            ["Show basic air quality data without AI analysis", "All Regions", 14, False]
        ],
        inputs=[prompt_input, region_input, days_input, ai_toggle]
    )

    # Connect button - NOW output_plot is defined
    analyze_btn.click(
        fn=analyze_air_quality,
        inputs=[prompt_input, region_input, days_input, ai_toggle],
        outputs=[output_text, output_plot]  # Now output_plot exists!
    )

# ==================== LAUNCH THE UI ====================
print("🚀 LAUNCHING AIR QUALITY DASHBOARD...")
print("⏳ Starting server...")

try:
    demo.launch(share=True, debug=True)
    print("✅ Dashboard is running! Check the URL above.")
except Exception as e:
    print(f"❌ Launch error: {e}")
    print("🔄 Trying alternative launch method...")
    demo.launch()